/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */


/* Q2: How many facilities do not charge a fee to members? */


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


/* Q9: This time, produce the same result as in Q8, but using a subquery. */


/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */



## Part 2 Questions

Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [7]:
# Import pandas, create_engine
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [16]:
with engine.connect() as con:
    rs = con.execute("""
    SELECT *
    FROM (
        SELECT f.name AS facility_name,
            SUM(
                CASE
                    WHEN b.memid = 0 THEN b.slots * f.guestcost
                    ELSE b.slots * f.membercost
                END) AS total_revenue
        FROM Bookings AS b
        INNER JOIN Facilities AS f
        USING (facid)
        GROUP BY name
        ORDER BY total_revenue DESC) AS result
    WHERE total_revenue < 1000;
    """)
    df = pd.DataFrame(rs.fetchall())
df

,facility_name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [20]:
with engine.connect() as con:
    rs = con.execute("""
	SELECT *
	FROM (
		SELECT m1.surname || ", " || m1.firstname AS member_name, 
			CASE
    			WHEN m2.recommendedby = 0 THEN 'None'
        		ELSE m2.surname || ', ' || m2.firstname
			END AS recommender
		FROM Members AS m1
		INNER JOIN Members AS m2
		ON m1.recommendedby = m2.memid
   		WHERE m1.memid > 0) AS result
    ORDER BY member_name;
    """)
    df = pd.DataFrame(rs.fetchall())
df

,member_name,recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


Q12: Find the facilities with their usage by member, but not guests

In [27]:
with engine.connect() as con:
    rs = con.execute("""
    SELECT name AS facility, member_name, SUM(slots) AS total_slots
    FROM Bookings AS b,
        (SELECT facid, name
         FROM Facilities) AS f,
        (SELECT memid, surname || ', ' || firstname AS member_name
        FROM Members) AS m
    WHERE b.facid = f.facid
        AND b.memid = m.memid
        AND b.memid > 0
    GROUP BY name, member_name
    ORDER BY name;
    """)
    df = pd.DataFrame(rs.fetchall())
df

,facility,member_name,num_bookings
0,Badminton Court,"Bader, Florence",27
1,Badminton Court,"Baker, Anne",30
2,Badminton Court,"Baker, Timothy",21
3,Badminton Court,"Boothe, Tim",36
4,Badminton Court,"Butters, Gerald",63
...,...,...,...
197,Tennis Court 2,"Smith, Darren",57
198,Tennis Court 2,"Smith, Jack",3
199,Tennis Court 2,"Smith, Tracy",6
200,Tennis Court 2,"Stibbons, Ponder",96


Q13: Find the facilities usage by month, but not guests

In [29]:
with engine.connect() as con:
    rs = con.execute("""
    SELECT strftime('%m', starttime) AS month, name AS facility, SUM(slots) AS total_slots
    FROM Bookings AS b,
        (SELECT facid, name
         FROM Facilities) AS f
    WHERE b.facid = f.facid
        AND memid > 0
    GROUP BY strftime('%m', starttime), name
    ORDER BY month, name;
    """)
    df = pd.DataFrame(rs.fetchall())
df

,month,facility,total_slots
0,07,Badminton Court,165
1,07,Massage Room 1,166
2,07,Massage Room 2,8
3,07,Pool Table,110
4,07,Snooker Table,140
5,07,Squash Court,50
6,07,Table Tennis,98
7,07,Tennis Court 1,201
8,07,Tennis Court 2,123
9,08,Badminton Court,414
